In [86]:
import urllib3
import numpy
import pandas as pd
import matplotlib
import seaborn
from datetime import datetime
import requests
import os
import re
print("All libs imported")

All libs imported


In [84]:
url = 'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php'
for i in range(1,28):
    params = {'country': 'UKR', 'provinceID': str(i), 'year1': '1981', 'year2': '2023', 'type': 'Mean'}
    response = requests.get(url, params=params)#, headers=headers)
    content = response.content
    content = content.replace(b'<tt><pre>', b'')
    content = content.replace(b'</pre></tt>', b'')
    now = datetime.now()
    date_and_time_time = now.strftime("%d%m%Y%H%M%S") 
    out = open(f"frames/obl_{i}_{date_and_time_time}.csv",'wb')
    out.write(content)
    out.close
print("Data downloaded")

Data downloaded


In [93]:
dir = "frames"
dflist = []
for filename in os.listdir(dir):
    province = re.search(r"obl_(\d+)_\d+", filename).group(1)
    filename = f"{dir}/{filename}"
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty'] 
    df = pd.read_csv(filename, header = 1, names = headers)
    df = df.drop(columns=['empty'])
    df['province'] = province
    dflist.append(df)

df=dflist[0]    
for dfi in dflist[1:]:
    df = pd.concat([df, dfi], ignore_index=True)
    
df


,Year,Week,SMN,SMT,VCI,TCI,VHI,province
30576,1982,1,0.061,253.43,17.93,90.95,54.44,23
30577,1982,2,0.059,255.61,16.96,85.31,51.13,23
30578,1982,3,0.060,258.56,16.90,74.56,45.73,23
30579,1982,4,0.060,261.17,16.45,63.52,39.98,23
30580,1982,5,0.061,263.92,15.86,53.34,34.60,23
...,...,...,...,...,...,...,...,...
32755,2023,48,-1.000,-1.00,-1.00,-1.00,-1.00,23
32756,2023,49,-1.000,-1.00,-1.00,-1.00,-1.00,23
32757,2023,50,-1.000,-1.00,-1.00,-1.00,-1.00,23
32758,2023,51,-1.000,-1.00,-1.00,-1.00,-1.00,23


In [95]:
# df[(df['province']=='23')]
text=""" 1: Cherkasy
2: Chernihiv
3: Chernivtsi
4: Crimea
5: Dnipropetrovs'k
6: Donets'k
7: Ivano-Frankivs'k
8: Kharkiv
9: Kherson
10: Khmel'nyts'kyy
11: Kiev
12: Kiev City
13: Kirovohrad
14: Luhans'k
15: L'viv
16: Mykolayiv
17: Odessa
18: Poltava
19: Rivne
20: Sevastopol'
21: Sumy
22: Ternopil'
23: Zakarpathia
24: Vinnytsya
25: Volyn
26: Zaporizhzhya
27: Zhytomyr"""
province_ids = {}
for line in text.split('\n'):
    if line.strip():
        value = line.split(':')[1].strip()
        key = line.split(':')[0].strip()
        province_ids[key] = value

print(province_ids)

{'1': 'Cherkasy', '2': 'Chernihiv', '3': 'Chernivtsi', '4': 'Crimea', '5': "Dnipropetrovs'k", '6': "Donets'k", '7': "Ivano-Frankivs'k", '8': 'Kharkiv', '9': 'Kherson', '10': "Khmel'nyts'kyy", '11': 'Kiev', '12': 'Kiev City', '13': 'Kirovohrad', '14': "Luhans'k", '15': "L'viv", '16': 'Mykolayiv', '17': 'Odessa', '18': 'Poltava', '19': 'Rivne', '20': "Sevastopol'", '21': 'Sumy', '22': "Ternopil'", '23': 'Zakarpathia', '24': 'Vinnytsya', '25': 'Volyn', '26': 'Zaporizhzhya', '27': 'Zhytomyr'}


In [96]:
df['province']=df['province'].map(province_ids)
df

,Year,Week,SMN,SMT,VCI,TCI,VHI,province
0,1982,1,0.059,258.24,51.11,48.78,49.95,Khmel'nyts'kyy
1,1982,2,0.063,261.53,55.89,38.20,47.04,Khmel'nyts'kyy
2,1982,3,0.063,263.45,57.30,32.69,44.99,Khmel'nyts'kyy
3,1982,4,0.061,265.10,53.96,28.62,41.29,Khmel'nyts'kyy
4,1982,5,0.058,266.42,46.87,28.57,37.72,Khmel'nyts'kyy
...,...,...,...,...,...,...,...,...
61147,2023,48,-1.000,-1.00,-1.00,-1.00,-1.00,Kherson
61148,2023,49,-1.000,-1.00,-1.00,-1.00,-1.00,Kherson
61149,2023,50,-1.000,-1.00,-1.00,-1.00,-1.00,Kherson
61150,2023,51,-1.000,-1.00,-1.00,-1.00,-1.00,Kherson


In [119]:
# selection task1 Ряд VHI для області за рік, пошук екстремумів (min та max); 
sel_prov="Kiev City"
sel_year=2023
selected=(df[(df['province']==sel_prov)&(df['Year']==sel_year)&(df['VHI']!=-1)][["Week","VHI"]])
min_vhi=selected.loc[selected['VHI'].idxmin()].to_dict()
max_vhi=selected.loc[selected['VHI'].idxmax()].to_dict()
print(f'Selected: Province = {sel_prov}, Year = {sel_year}\n\n',selected)
print(f'\nMIN VHI: {min_vhi}\nMAX VHI: {max_vhi} ')

Selected: Province = Kiev City, Year = 2023

       Week    VHI
6500     1  38.50
6501     2  39.52
6502     3  41.44
6503     4  46.18
6504     5  47.89
6505     6  47.72
6506     7  47.18
6507     8  48.47

MIN VHI: {'Week': 1.0, 'VHI': 38.5}
MAX VHI: {'Week': 8.0, 'VHI': 48.47} 


In [178]:
# selection Task 2 Ряд VHI за всі роки для області, виявити роки з екстремальними  посухами, які торкнулися більше вказаного відсотка області;
sel_prov="Kiev City"
selected=(df[(df['province']==sel_prov)&(df['VHI']!=-1)][['Year',"Week","VHI"]])
print(f'Selected Province: {sel_prov}\n',selected.head().to_string(index=False))
# Процент сильних посух в кожному році
percent_extreme_vhi_by_year=selected.groupby('Year')['VHI'].apply(lambda x: (x < 15).mean()*100)
# print(percent_extreme_vhi_by_year)
# Вказаний процент 
year_percent = 15 
extreme_vhi=percent_extreme_vhi_by_year.apply(lambda x: f"extreme VHI(%): {x:.1f}" if x > year_percent else None)
print(f"\nРоки з екстримальними посухами: (% екстримальних посух в році > {year_percent}%)\n",extreme_vhi.dropna().to_string())


Selected Province: Kiev City
  Year  Week   VHI
 1982     1 44.85
 1982     2 40.95
 1982     3 39.52
 1982     4 39.39
 1982     5 41.34

Роки з екстримальними посухами: (% екстримальних посух в році > 15%)
 Year
2000    extreme VHI(%): 19.2


In [181]:
# selection task3 Аналогічно для помірних посух
# Процент помірних посух в кожному році
percent_middle_vhi_by_year=selected.groupby('Year')['VHI'].apply(lambda x: ((x > 15)&(x < 35)).mean()*100)
# Вказаний процент 
year_percent = 20
middle_vhi=percent_middle_vhi_by_year.apply(lambda x: f"middle VHI(%): {x:.1f}" if x > year_percent else None)
print(f"\nРоки з помірними посухами: (% помірних посух в році > {year_percent}%)\n",middle_vhi.dropna().to_string())



Роки з помірними посухами: (% помірних посух в році > 20%)
 Year
1985    middle VHI(%): 20.5
1986    middle VHI(%): 40.4
1989    middle VHI(%): 26.9
1992    middle VHI(%): 26.9
1995    middle VHI(%): 24.5
2000    middle VHI(%): 23.1
2009    middle VHI(%): 21.2
2010    middle VHI(%): 26.9


# Контрольні запитання 

1. Основні кроки по вирішенню задачі аналізу даних 
     Визначити постановку задачі;
     Визначити ідеальні дані;
     Визначити, до яких даних є доступ (реальні дані);
     Зібрати дані;
     Підготувати дані для використання;
     Початковий аналіз (exploratory analysis);
     Аналіз статистики; моделювання, прогнозування;
     Інтерпретація результатів;
     Опис виявлених законів, залежностей;
     Написання коду для повторюваності дослідження;
     Обмін результатами
    
    Можливі джерела даних
         Дані можуть вільно поширюватись через Інтернет;
         Можна придбати якісні дані, які є результатом інтелектуальної праці
        деякої групи експертів;
         Якщо потрібних даних не існує, використати модельні дані (застосовне
        лише за відсутності НЕштучних даних або при їх жорсткому дефіциті)
        (NN GANs);

         Важливо: поважати умови надання доступу до даних (Terms of use) +
        права інтелектуальної власності
    
    Отримання даних
         Спробувати отримати «сирі дані» (raw data), які не містять результатів стороннього втручання;
         Не забути зафіксувати джерело даних та, при необхідності,
        проінформувати власника даних про їх використання (якщо це вимагається в Terms of Use);
         У випадку отримання даних з Інтернет зафіксувати (зберегти) URL- адресу 
        та момент часу, коли дані були отримані
        
    Очистка даних (фільтрація)
         «Сирі дані» (raw data) часто потребують попередньої обробки перед
        подальшим застосуванням. Це може стати справжньою проблемою
         Якщо «сирі дані» не доступні, а надаються лише результати їх обробки,
        необхідно з'ясувати, яким чином була здійснена попередня обробка;
         Зрозуміти природу джерела даних;
         Виділення коректної вибірки, яка задовольняє вимогам дослідження,
        «переформатування» даних;
         Визначення якості даних: використовувати дані надалі або відкинути і
        пошукати якісніше джерело?

    Exploratory data analysis
         Аналіз набору даних загалом: середнє, min, max, кількість спостережень
         Виявлення відсутніх даних (missing data, NA);
         Створення простих графіків для оцінки якості даних (exploratory plots);
         Попередній аналіз (напр., кластеризація, пошук явних залежностей між інформаційними параметрами)
        
        
2. В чому переваги Python над іншими мовами програмування у сфері  вирішення задач Data Science? 
    Відносно проста і зрозуміла мова для розробників
    кросплатформенний код
    Можливість інтерактивної роботи(IPython,IPython Notebook)
    BSD-ліцензія
    Велике comminuty
    Якість:рівень помилок - найнижчий для open source продуктів
    
3. Дати визначення поняттю «ідеальний набір даних» 

    Ідеальний набір даних у Data science - це набір даних, який містить всі необхідні дані для вирішення 
    конкретної задачі, зібраних відповідно до належних методологій та стандартів збору даних, з мінімальним 
    рівнем шуму та відсутністю пропущених значень.

        Ідеальний набір даних повинен відповідати наступним критеріям:

        Якість: дані повинні бути достовірними, точними та надійними.

        Кількість: дані повинні бути достатньою кількістю для вирішення задачі.

        Розмаїтість: набір даних повинен містити різноманітні дані, щоб дозволити моделі бути  точнішою.

        Повнота: набір даних повинен містити всі необхідні дані для вирішення задачі, щоб уникнути необхідності в 
        додатковому зборі даних.

    Наявність: набір даних повинен бути доступним та легко збирається.
    
4. Які, на Вашу думку, існують джерела даних? 

   Можливі джерела даних
         Дані можуть вільно поширюватись через Інтернет;
         Можна придбати якісні дані, які є результатом інтелектуальної праці
        деякої групи експертів;
         Якщо потрібних даних не існує, використати модельні дані (застосовне
        лише за відсутності НЕштучних даних або при їх жорсткому дефіциті)
        (NN GANs);

         Важливо: поважати умови надання доступу до даних (Terms of use) +
        права інтелектуальної власності
    
5. Які дані можна вважати «чистими» (clean data)?  
    "Чисті" дані, це дані, які мають високу якість та достовірність, тобто дані, які не містять помилок, пропущених 
    значень або неточностей. Такі дані зазвичай забезпечують більш точний та надійний аналіз.